In [11]:
from functools import *
import numpy as np
import matplotlib.pyplot as plt
import scipy as sci

from defining_the_hamiltonian import fermi_hubbard_hamiltonian, swap_electron, number_up_down

from qiskit.circuit.library import EfficientSU2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.transpiler import PassManager

from qiskit_ibm_runtime import QiskitRuntimeService, Session
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from qiskit_ibm_runtime.fake_provider import FakeMelbourneV2
from qiskit_ibm_runtime.fake_provider import FakeSherbrooke
from qiskit_aer import AerSimulator

from qiskit import QuantumCircuit
from qiskit.visualization import plot_histogram
from qiskit_ibm_runtime.sampler import SamplerV2
from qiskit import transpile

from qiskit.quantum_info.operators import SparsePauliOp
from qiskit.circuit.library import n_local
from qiskit.quantum_info.operators import Pauli
from qiskit.circuit.library import RealAmplitudes
from qiskit.circuit.library import EfficientSU2
from qiskit.circuit.library import excitation_preserving

from qiskit_nature.second_q.circuit.library import UCC
from qiskit_nature.second_q.mappers import JordanWignerMapper
from qiskit_nature.second_q.hamiltonians import FermiHubbardModel
from qiskit_nature.second_q.hamiltonians.lattices import Lattice

from qiskit_aer.primitives import EstimatorV2


In [12]:
backend = AerSimulator()
# sherbrooke = FakeSherbrooke()
# backend = AerSimulator.from_backend(sherbrooke)


In [ ]:
nodes = 6
weighted_edges = [
    (0,1,1),(0,2,1),(0,3,1),(0,4,1),
    (5,1,1),(5,2,1),(5,3,1),(5,4,1),
    (1,2,1),(2,3,1),(3,4,1),(4,1,1)
    ]

octahedron = Lattice.from_nodes_and_edges(num_nodes=nodes, weighted_edges=weighted_edges)
lattice = octahedron.uniform_parameters(
    uniform_interaction=-1,uniform_onsite_potential=0)
fermi_hubb = FermiHubbardModel(octahedron, onsite_interaction=1)

In [14]:
hamiltonian = JordanWignerMapper().map(fermi_hubb.second_q_op())

qc = QuantumCircuit(12)
qc.x(0)
qc.x(6)

ansatz = qc.compose(excitation_preserving(nodes*2, reps=2, 
                                          insert_barriers=True,
                                          skip_final_rotation_layer=False,
                                          entanglement = "pairwise",
                                          mode = "iswap"))

ansatz.draw("mpl")

num_of_parameters = ansatz.num_parameters

print(f"There are {num_of_parameters} parameters.")
# ansatz.decompose().draw("mpl", style="iqp")

There are 58 parameters.


In [15]:

pm:PassManager = generate_preset_pass_manager(backend=backend, optimization_level=3)

ansatz_isa = pm.run(ansatz)
hamiltonian_isa = hamiltonian.apply_layout(layout=ansatz_isa.layout)


In [16]:
cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")

    return energy

In [17]:
x0 = np.array([ 3.81753823,  4.82121225,  4.67535109,  3.0675717 ,  3.46028147,
        2.2842241 ,  2.28068808,  2.93173167,  4.45084888,  3.59009141,
        6.22908827,  0.69924701,  4.02079642,  1.27397888,  4.55890384,
        1.8248962 ,  1.50390195,  4.79395989,  0.98523768,  6.11700762,
        0.13819355,  7.01209955,  5.33633725,  0.65456489,  1.20210642,
        0.83180232,  5.82866698,  5.19575042,  3.31176441,  5.50647245,
        2.76759528,  2.94891426,  3.90298021,  5.33175021,  4.30725485,
        5.01166516,  1.24604705,  3.44888302,  5.73396634,  5.35505843,
       -0.28159186,  4.28582381,  5.46612932, -0.11433386,  4.83048706,
        4.14032701,  2.36388634,  4.38384209,  6.34758701,  7.05600274,
        5.77834296,  4.9689087 ,  3.62937989,  5.05953697,  3.79919937,
        1.58854063,  3.4267587 ,  7.68951643])
len(x0)

58

In [20]:
estimator = Estimator(backend)
pub = (ansatz_isa, [hamiltonian_isa], x0)
result = estimator.run(pubs=[pub]).result()
energy = result[0].data.evs[0]
print(energy)

3.185546875


In [19]:
# with Session(backend=backend) as session:
#     estimator = Estimator(mode=session)
#     estimator.options.default_shots = 4000

#     res = sci.optimize.minimize(
#         cost_func,
#         x0,
#         args=(ansatz_isa, hamiltonian_isa, estimator),
#         method="cobyla",
#     )
# print("done")